# Курсовая работа


## Выполнил студент группы БФИ2204 Момотенко Кирилл Витальевич
***

### Содержание кода

In [45]:
import csv
from openpyxl import Workbook
from openpyxl.utils.exceptions import InvalidFileException

# Чтение данных из CSV файла
def read_csv(file_name):
    sales_data = []
    try:
        with open(file_name, newline='') as csvfile:
            csvreader = csv.reader(csvfile, delimiter='|')
            next(csvreader)  # пропускаем заголовок
            for row in csvreader:
                try:
                    sales_data.append({
                        'order_number': row[0],
                        'order_date': row[1],
                        'product_name': row[2],
                        'category': row[3],
                        'quantity_sold': int(row[4]),
                        'unit_price': float(row[5]),
                        'total_price': float(row[6])
                    })
                except ValueError as e:
                    print(f"Ошибка обработки строки {row}: {e}")
    except FileNotFoundError:
        print(f"Файл {file_name} не найден.")
    except csv.Error as e:
        print(f"Ошибка чтения CSV файла: {e}")
    return sales_data

#Общая выручка магазина
def calculate_total_revenue(sales_data):
    return sum([sale['total_price'] for sale in sales_data])

#Товар, который был продан наибольшее количество раз
def find_most_sold_product(sales_data):
    products_sold = {}
    for sale in sales_data:
        if sale['product_name'] in products_sold:
            products_sold[sale['product_name']] += sale['quantity_sold']
        else:
            products_sold[sale['product_name']] = sale['quantity_sold']
    return max(products_sold, key=products_sold.get) if products_sold else None

#Товар, который принес наибольшую выручку
def find_highest_revenue_product(sales_data):
    revenue_per_product = {}
    for sale in sales_data:
        if sale['product_name'] in revenue_per_product:
            revenue_per_product[sale['product_name']] += sale['total_price']
        else:
            revenue_per_product[sale['product_name']] = sale['total_price']
    return max(revenue_per_product, key=revenue_per_product.get) if revenue_per_product else None
#Отчет
def generate_report(sales_data):
    total_revenue = calculate_total_revenue(sales_data)
    products_sold = {}
    for sale in sales_data:
        if sale['product_name'] in products_sold:
            products_sold[sale['product_name']] += sale['quantity_sold']
        else:
            products_sold[sale['product_name']] = sale['quantity_sold']
    
    report = f"Общий доход: {total_revenue} руб.\n\nПродукты:\n"
    for product, quantity_sold in products_sold.items():
        revenue_share = (quantity_sold * sales_data[0]['unit_price']) / total_revenue
        report += f"{product}: {quantity_sold} шт., Отношение продаж: {revenue_share:.2%}\n"

    return report

def write_to_excel(sales_data, report):
    wb = Workbook()
    ws = wb.active

    try:
        ws.append(["Отчет по продажам"])
        ws.append(["Общий доход", calculate_total_revenue(sales_data)])

        ws.append([])

        ws.append(["Номер заказа", "Дата заказа", "Название продукта", "Категория", "Количество продано", "Цена за единицу", "Общая цена"])

        for sale in sales_data:
            ws.append([
                sale['order_number'],
                sale['order_date'],
                sale['product_name'],
                sale['category'],
                sale['quantity_sold'],
                sale['unit_price'],
                sale['total_price']
            ])

        ws.append([])

        ws.append(["Продукт", "Количество продано", "Отношение продаж"])

        products_sold = {}
        for sale in sales_data:
            if sale['product_name'] in products_sold:
                products_sold[sale['product_name']] += sale['quantity_sold']
            else:
                products_sold[sale['product_name']] = sale['quantity_sold']

        total_revenue = calculate_total_revenue(sales_data)
        for product, quantity_sold in products_sold.items():
            revenue_share = (quantity_sold * sales_data[0]['unit_price']) / total_revenue
            ws.append([product, quantity_sold, f"{revenue_share:.2%}"])

        wb.save('report.xlsx')
    except InvalidFileException as e:
        print(f"Ошибка записи в Excel файл: {e}")
    except PermissionError:
        print("Ошибка записи в файл: отказано в доступе.")
    except Exception as e:
        print(f"Неизвестная ошибка при записи в Excel файл: {e}")

if __name__ == "__main__":
    file_name = 'sales_data.csv'
    sales_data = read_csv(file_name)

    if not sales_data:
        print("Нет данных для обработки.")
    else:
        most_sold_product = find_most_sold_product(sales_data)
        if most_sold_product:
            print(f"Самый продаваемый продукт: {most_sold_product}")
        else:
            print("Не удалось определить самый продаваемый продукт.")

        highest_revenue_product = find_highest_revenue_product(sales_data)
        if highest_revenue_product:
            print(f"Самый доходный продукт: {highest_revenue_product}")
        else:
            print("Не удалось определить самый доходный продукт.")

        report = generate_report(sales_data)
        print(report)

        write_to_excel(sales_data, report)

Самый продаваемый продукт:  Очки Ray-Ban 
Самый доходный продукт:  Очки Ray-Ban 
Общий доход: 32050.0 руб.

Продукты:
 Сумка Chanel : 8 шт., Отношение продаж: 14.98%
 Очки Ray-Ban : 10 шт., Отношение продаж: 18.72%
 Платье Dior  : 7 шт., Отношение продаж: 13.10%
 Кеды Gucci   : 5 шт., Отношение продаж: 9.36%
 Очки sun-rise: 7 шт., Отношение продаж: 13.10%
 Шарф Сhanel  : 9 шт., Отношение продаж: 16.85%

